In [1]:
import os

In [2]:
%pwd

'c:\\Users\\patel\\End-to-end-ML-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\patel\\End-to-end-ML-Project'

In [5]:
#entity- its a return type of a functn, where configbox is prebuilt entity  coming from box where alrdy created wht is the return type
#here we r creating custom return type
# where data ingstn only return rootdir,url,datafile,unzipdir the thing whch 
#  r mentnd inside config.yaml
from dataclasses import dataclass  #is importng to create custom entity
from pathlib import Path

# below wrttn is custom entity whr 1st mntn decorator
@dataclass(frozen=True)    #using dataclss write decorator on top of clss & it will cnvrt to entity
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
#importng constnt cze inside it we hv all the variables to read
from mlProject.constants import * # * means evrythng
from mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
        

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
# updating componets

import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [9]:
class DataIngestion:    #creting clss clld data ingestn
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self): #inside clss creting mthd clld download file to dwnlod
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(      #using rqst package
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self):     #to extract zip file
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
# to execute cll pipline created below
#pplne means whch functn to cll 1st, steps
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-10-20 16:04:14,381: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-20 16:04:14,386: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-20 16:04:14,390: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-10-20 16:04:14,392: INFO: common: created directory at: artifacts]
[2023-10-20 16:04:14,402: INFO: common: created directory at: artifacts/data_ingestion]
[2023-10-20 16:04:16,738: INFO: 1543902713: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 22
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "f77c82398ffde79ce2f18a081012da49a848d6ad8197e282241cf2593793168b"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 9C76:519D:1B6E37:21AFFA:653257A8
Accept-Ranges: bytes
Date: Fri, 20 Oct 